# Simple Observability 

_Created by Eumar Assis (eumar.assis@databricks.com_)

This notebook computes and consolidates observability metrics for the Lakehouse into an unified Delta table, making it easy to build operational dashboards. It provides the following insights:

- Inventory of all tables, including their format and storage location
- Table size and growth over time
- Data Freshness: tables change dates
- Last optimization and vacuum dates



**How does it work?** The notebook queries the [system.information_schema](https://docs.databricks.com/en/sql/language-manual/sql-ref-information-schema.html) table based on the provided regular expression to find the desired tables. Then, it uses [DESCRIBE DETAIL](https://docs.databricks.com/en/delta/table-details.html) and [DESCRIBE HISTORY](https://docs.databricks.com/en/sql/language-manual/delta-describe-history.html) for each table to collect the appropriate metrics. The notebook uses a temporary table to make the code idempotent. It will only replace the records in the destination table at the end with an atomic MERGE.

Based on initial tests, the notebook script was able to process 314 tables in 40 minutes by using a 3-nodes cluster with 32GB nodes. Choosing a larger master node and more workers is likely to improve the performance and run the script faster.

**Requirements:**

- User running this notebook will need the USE CATALOG, USE SCHEMA, and SELECT permissions on tables targeted for the script.

**Parameters:**

- **destination_table**: provide the name of the destination table, must use a three-level namespace. e.g. catalog_name.schema_name.table_name
- **filter**: regular expression to filter which catalogs, schemas, or tables the metrics should be collected for. Examples:
  - All schemas and tables under catalog 'demo': `^demo\.`
  - All tables under catalog 'demo' and schema 'dbdemo': `^demo\.dbdemo\.`
  - Single table 'tabledemo': `demo.dbdemo.tabledemo`

**Output Table:**


| Column Name             | Type       | Description                            |
|-------------------------|------------|----------------------------------------|
| `timestamp`             | TIMESTAMP  | The timestamp of the record            |
| `catalogName`           | STRING     | The name of the catalog                |
| `schemaName`            | STRING     | The name of the schema                 |
| `tableName`             | STRING     | The name of the table                  |
| `creationDate`          | TIMESTAMP  | The creation date of the table         |
| `createdBy`             | STRING     | The creator of the table               |
| `owner`                 | STRING     | The owner of the table                 |
| `type`                  | STRING     | The type of the table                  |
| `format`                | STRING     | The format of the table                |
| `storageLocation`       | STRING     | The storage location of the table      |
| `lastVacuumDate`        | TIMESTAMP  | The date of the last vacuum operation  |
| `lastOptimizeDate`      | TIMESTAMP  | The date of the last optimize operation|
| `lastSchemaChangeDate`  | TIMESTAMP  | The date of the last schema change     |
| `lastWriteDate`         | TIMESTAMP  | The date of the last write operation   |
| `sizeInMB`              | DOUBLE     | The size of the table in MB            |
| `sizeInMB24Hours`       | DOUBLE     | The size of the table in the last 24 hours |
| `sizeInMB7Days`         | DOUBLE     | The size of the table in the last 7 days   |
| `sizeInMb30Days`        | DOUBLE     | The size of the table in the last 30 days  |
| `partitionColumns`      | STRING     | The partition columns of the table     |
| `clusteringColumns`     | STRING     | The clustering columns of the table    |
| `sizeHistory`           | STRING     | This size history, storing a JSON array     |


In [0]:

dbutils.widgets.text("filter", "^catalogName\.", "Table Filter Regex")

dbutils.widgets.text("destination_table", "eumar_tests.eumar_default.simple_observability", "Destination table")

In [0]:
from datetime import datetime

# Get the destination table name from the widget
destination_table = dbutils.widgets.get("destination_table")

temporary_table = destination_table + "_temp_processing_" + datetime.now().strftime('%Y%m%d%H')

# Creates the destination Delta table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {destination_table}
(
  timestamp TIMESTAMP,
  catalogName STRING,
  schemaName STRING,
  tableName STRING,
  creationDate TIMESTAMP,
  createdBy STRING,
  owner STRING,
  type STRING,
  format STRING,
  storageLocation STRING,
  lastVacuumDate TIMESTAMP,
  lastOptimizeDate TIMESTAMP,
  lastSchemaChangeDate TIMESTAMP,
  lastWriteDate TIMESTAMP,
  sizeInMB DOUBLE,
  sizeInMB24Hours DOUBLE,
  sizeInMB7Days DOUBLE,
  sizeInMb30Days DOUBLE,
  partitionColumns STRING,
  clusteringColumns STRING,
  sizeHistory STRING  -- This column will store the JSON data
)
USING delta
""")

# Create a temporary table with the same schema as the original table
spark.sql(f"SELECT * FROM {destination_table} LIMIT 0").write.mode("overwrite").saveAsTable(temporary_table)



To find the targeted tables, we will query and filter the system.information_schema table using the Regex. Then, we'll insert the target tables into the temporary table.
https://docs.databricks.com/en/sql/language-manual/sql-ref-information-schema.html

In [0]:
%sql
select distinct table_type from system.information_schema.tables

table_type
VIEW
STREAMING_TABLE
MANAGED_SHALLOW_CLONE
MATERIALIZED_VIEW
EXTERNAL
FOREIGN
MANAGED
EXTERNAL_SHALLOW_CLONE


In [0]:
# Get the value of the widget
table_filter_regex = dbutils.widgets.get("filter")

# Define query to insert filtered results into the temp table
sql_query = f"""
INSERT INTO {temporary_table} 
  (
    timestamp,
    catalogName,
    schemaName,
    tableName,
    creationDate,
    createdBy,
    owner,
    lastSchemaChangeDate,    
    type,
    format,
    sizeHistory
  )
SELECT
  CURRENT_DATE(),
  table_catalog,
  table_schema,
  table_name,
  created,
  created_by,  
  table_owner,
  last_altered,
  table_type,
  data_source_format,
  to_json(array())

FROM system.information_schema.tables
WHERE CONCAT(table_catalog, '.', table_schema, '.', table_name) RLIKE r'{table_filter_regex}'
AND table_owner != 'System user' AND table_type IN ('MANAGED', 'EXTERNAL' )
"""

# Execute the SQL query
spark.sql(sql_query)

display(spark.sql(f"SELECT COUNT(*) as records_to_process FROM {temporary_table}"))


records_to_process
250


In [0]:
from datetime import datetime
from pyspark.sql.functions import col
import json

# Define a function to filter and get the first timestamp for a given set of operations
def get_first_timestamp(operations, history_array):
    filtered = [row["timestamp"] for row in history_array if row["operation"] in operations]
    return filtered[0] if filtered else None
  
# this cannot be an UDF since UDFs cannot access the spark context
def get_update_history(catalog, schema, table, table_format):

  details = spark.sql(f"DESCRIBE DETAIL `{catalog}`.`{schema}`.`{table}`").collect()[0]

  last_vacuum_date = None
  last_optimize_date = None
  last_write_date = None

  if table_format == "DELTA":
    history = spark.sql(f"DESCRIBE HISTORY `{catalog}`.`{schema}`.`{table}`").orderBy("timestamp", ascending=False).collect()
    
    #Extract the required columns
    # vacuum_date_row = history.filter(col("operation").isin(["VACUUM"])).select("timestamp").first()
    # optimize_date_row = history.filter(col("operation").isin(["OPTIMIZE"])).select("timestamp").first()
    # last_write_date_row = history.filter(col("operation").isin(["WRITE", "MERGE", "DELETE", "UPDATE"])).select("timestamp").first()

    last_vacuum_date = get_first_timestamp(["VACUUM"], history)
    last_optimize_date = get_first_timestamp(["OPTIMIZE"], history)
    last_write_date = get_first_timestamp(["WRITE", "MERGE", "DELETE", "UPDATE"], history)

  size_in_mb = 0 if details["sizeInBytes"] is None else details["sizeInBytes"] / (1024 * 1024)
  partition_columns = ",".join(details["partitionColumns"]) if details["partitionColumns"] is not None else ""
  clustering_columns = ",".join(details["clusteringColumns"]) if details["clusteringColumns"] is not None else ""
  storage_location = details["location"]


  return {
        "lastVacuumDate": last_vacuum_date,
        "lastOptimizeDate": last_optimize_date,
        "lastWriteDate": last_write_date,
        "sizeInMB": float(size_in_mb),
        "storageLocation": storage_location,
        "partitionColumns": partition_columns,
        "clusteringColumns": clustering_columns,
        "sizeHistory" : json.dumps([{
          "size" : float(size_in_mb),
          "date" : datetime.now().strftime('%Y-%m-%d')
        }])
  }


In [0]:
# # Get the DataFrame
# df = spark.sql(f"SELECT * FROM {temporary_table}")

# # Create a new list to store the updated rows
# updated_rows = []

# rows_processed = 0

# rows_collect = df.collect()

# print (f"Getting history for tables. Total tables: {len(rows_collect)}")

# # Iterate over the rows and update the columns
# for row in rows_collect:
#     try:
#       update_history = get_update_history(row['catalogName'], row['schemaName'], row['tableName'], row['format'])

#       updated_row = row.asDict()
      
#       for key in update_history.keys():
#           updated_row[key] = update_history[key]
      
#       updated_rows.append(updated_row)

#       rows_processed = rows_processed + 1

#       print(f"({rows_processed}) {row['catalogName']}.{row['schemaName']}.{row['tableName']} completed")
#     except Exception as e:
#       # Code to handle any other exceptions
#       print(f"{row['catalogName']}.{row['schemaName']}.{row['tableName']} failed")
#       print("An error occurred:", str(e)) 
    
# # Create a new DataFrame with the updated rows
# updated_df = spark.createDataFrame(updated_rows, df.schema)

# # Save the changes in bulk
# updated_df.write.mode("overwrite").insertInto(temporary_table)

In [0]:
import concurrent.futures

# Get the DataFrame
df = spark.sql(f"SELECT * FROM {temporary_table}")

# Create a new list to store the updated rows
updated_rows = []

rows_processed = 0

rows_collect = df.collect()

print(f"Getting history for tables. Total tables: {len(rows_collect)}")

# Define a function to process each row and update the columns
def process_row(row):
    try:
        update_history = get_update_history(row['catalogName'], row['schemaName'], row['tableName'], row['format'])

        updated_row = row.asDict()

        for key in update_history.keys():
            updated_row[key] = update_history[key]

        return updated_row

    except Exception as e:
        # Code to handle any other exceptions
        print(f"{row['catalogName']}.{row['schemaName']}.{row['tableName']} failed")
        print("An error occurred:", str(e))

# Create a ThreadPoolExecutor with a maximum of 5 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Submit each row to the executor for processing
    futures = [executor.submit(process_row, row) for row in rows_collect]

    # Iterate over the completed futures and get the updated rows
    for future in concurrent.futures.as_completed(futures):
        updated_row = future.result()
        if updated_row:
            updated_rows.append(updated_row)
            rows_processed += 1
            print(f"({rows_processed}) {updated_row['catalogName']}.{updated_row['schemaName']}.{updated_row['tableName']} completed")

# Create a new DataFrame with the updated rows
updated_df = spark.createDataFrame(updated_rows, df.schema)

# Save the changes in bulk
updated_df.write.mode("overwrite").insertInto(temporary_table)

Getting history for tables. Total tables: 250
(1) dbdemos.fsi_credit_decisioning_sergio.fund_trans_gold_features completed
(2) dbdemos.dbdemos_iot_turbine.turbine_training_dataset_ml completed
(3) dbdemos.dbdemos_retail_c360_lhb.ml_churn_features completed
(4) dbdemos.lakehouse_monitor.wine_udit_an_profile_metrics completed
(5) dbdemos.retail_tayo_akigbogun.user_updates completed
(6) dbdemos.lakehouse_c360.churn_features_drift_metrics completed
(7) dbdemos.rag_chatbot_julia_hofmeister_swn_.databricks_pdf_documentation completed
(8) dbdemos.rag_chatbot_prabhjot_singh_kirby3.databricks_pdf_documentation completed
(9) dbdemos.danhls.cohort completed
(10) dbdemos.retail_amine_elhelou.mlops_churn_inference_log_baseline completed
(11) dbdemos.retail_raghuraj_vemula.user_clustering completed
(12) dbdemos.retail_amine_elhelou.mlops_churn_shap_importance completed
(13) dbdemos.data_talks.abc_profile_metrics completed
(14) dbdemos.retail_gabe_rybeck.user_updates completed
(15) dbdemos.fsi_credit

In [0]:
sql_query = f"""
MERGE INTO {destination_table} AS d
USING {temporary_table} AS i
ON d.catalogName = i.catalogName
   AND d.schemaName = i.schemaName
   AND d.tableName = i.tableName
WHEN MATCHED THEN
  UPDATE SET
    d.timestamp = CURRENT_DATE(),
    d.creationDate = i.creationDate,
    d.createdBy = i.createdBy,
    d.lastSchemaChangeDate = i.lastSchemaChangeDate,
    d.owner = i.owner,
    d.type = i.type,
    d.format = i.format,
    d.sizeInMB = i.sizeInMB,
    d.storageLocation = i.storageLocation,
    d.lastVacuumDate = i.lastVacuumDate,
    d.lastOptimizeDate = i.lastOptimizeDate,
    d.lastWriteDate = i.lastWriteDate,
    d.partitionColumns = i.partitionColumns,
    d.clusteringColumns = i.clusteringColumns,
    d.sizeHistory = to_json(array_union(from_json(d.sizeHistory, 'array<struct<date:string,size:double>>'),
                                        from_json(i.sizeHistory, 'array<struct<date:string,size:double>>')))
WHEN NOT MATCHED THEN
  INSERT (
    timestamp,
    catalogName,
    schemaName,
    tableName,
    creationDate,
    createdBy,
    lastSchemaChangeDate,
    owner,
    type,
    format,
    sizeInMB,
    storageLocation,
    lastVacuumDate,
    lastOptimizeDate,
    lastWriteDate,
    partitionColumns,
    clusteringColumns,    
    sizeHistory
  )
  VALUES (
    CURRENT_DATE(),
    i.catalogName,
    i.schemaName,
    i.tableName,
    i.creationDate,
    i.createdBy,
    i.lastSchemaChangeDate,
    i.owner,
    i.type,
    i.format,
    i.sizeInMB,
    i.storageLocation,
    i.lastVacuumDate,
    i.lastOptimizeDate,
    i.lastWriteDate,
    i.partitionColumns,
    i.clusteringColumns,    
    i.sizeHistory
  )
"""

display(spark.sql(sql_query))

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
250,0,0,250


In [0]:

sql_query = f"""
MERGE INTO {destination_table} AS target
USING (
  WITH exploded_table AS (
      SELECT 
          t.catalogName,
          t.schemaName,
          t.tableName,
          explode(from_json(t.sizeHistory, 'array<struct<date:string, size:double>>')) AS sizeHistoryElement
      FROM 
          {destination_table} t
      INNER JOIN
        {temporary_table} s
      ON t.catalogName = s.catalogName AND t.schemaName = s.schemaName AND t.tableName = s.tableName
      ORDER BY sizeHistoryElement.date DESC          
  )
  SELECT  
      catalogName,
      schemaName,
      tableName,
      MAX(CASE WHEN date_diff(current_date(), sizeHistoryElement.date) >= 1 THEN sizeHistoryElement.size ELSE NULL END) AS sizeInMB24Hours,
      MAX(CASE WHEN date_diff(current_date(), sizeHistoryElement.date) >= 7 THEN sizeHistoryElement.size ELSE NULL END) AS sizeInMB7Days,
      MAX(CASE WHEN date_diff(current_date(), sizeHistoryElement.date) >= 30 THEN sizeHistoryElement.size ELSE NULL END) AS sizeInMB30Days
  FROM 
      exploded_table
  GROUP BY
      catalogName,
      schemaName,
      tableName
) AS source
ON target.catalogName = source.catalogName AND target.schemaName = source.schemaName AND target.tableName = source.tableName
WHEN MATCHED THEN
  UPDATE SET
    target.sizeInMB24Hours = source.sizeInMB24Hours,
    target.sizeInMB7Days = source.sizeInMB7Days,
    target.sizeInMb30Days = source.sizeInMb30Days;
"""

display(spark.sql(sql_query))

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
250,250,0,0


In [0]:
query = f"""
SELECT 
  t.* 
FROM 
  {destination_table} t
INNER JOIN
  {temporary_table} s
  ON t.catalogName = s.catalogName AND t.schemaName = s.schemaName AND t.tableName = s.tableName 
"""

display(spark.sql(query))

timestamp,catalogName,schemaName,tableName,creationDate,createdBy,owner,type,format,storageLocation,lastVacuumDate,lastOptimizeDate,lastSchemaChangeDate,lastWriteDate,sizeInMB,sizeInMB24Hours,sizeInMB7Days,sizeInMb30Days,partitionColumns,clusteringColumns,sizeHistory
2024-06-03T00:00:00Z,dbdemos,tamu,model_metadata,2024-03-01T16:15:25.865Z,alexander.metrocavich@databricks.com,alexander.metrocavich@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/b4496252-7499-4319-8319-1bb55a9ab957,null,null,2024-03-01T16:15:25.865Z,null,0.019945144653320312,null,null,null,,,"[{""size"": 0.019945144653320312, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,dbdemos_iot_turbine,turbine_drift_metrics,2024-06-02T17:14:00.64Z,kapil.budhraja@databricks.com,kapil.budhraja@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/2dbaa64a-da94-474d-b3d4-5e442ab9bfd6,null,null,2024-06-02T17:14:00.64Z,null,0.006602287292480469,null,null,null,,,"[{""size"": 0.006602287292480469, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,hls_patient_readmission,conditions_ml,2024-01-11T04:16:34.701Z,nitin.aggarwal@databricks.com,account users,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/df4501c0-ac56-43e7-860d-dd4f62aea208,null,null,2024-05-16T14:19:34.459Z,null,38.99062156677246,null,null,null,,,"[{""size"": 38.99062156677246, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,hls_patient_readmission,patient_readmission_prediction_profile_metrics,2024-06-01T22:48:51.639Z,neha.pande@databricks.com,account users,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3f845d98-cd56-408a-bb5b-3a743c04f0eb,null,null,2024-06-03T16:39:15.041Z,2024-06-02T22:48:40Z,0.0,null,null,null,,,"[{""size"": 0.0, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,fsi_wm_nba,bronze_dim_funds_llm_profile_metrics,2024-05-30T17:24:52.866Z,annemarie.ou@databricks.com,annemarie.ou@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/80aba3c4-9fe9-457e-b273-bbe5b1c3802d,null,null,2024-05-30T17:24:52.866Z,2024-06-03T10:23:41Z,0.0,null,null,null,,,"[{""size"": 0.0, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,rag_chatbot_julia_hofmeister_swn_energy,pdf_raw,2024-05-13T21:15:27.185Z,julia.hofmeister@databricks.com,julia.hofmeister@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/72749965-14fb-4f39-88d9-869a55f0a489,null,null,2024-05-13T21:15:27.185Z,null,2.3991756439208984,null,null,null,,,"[{""size"": 2.3991756439208984, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,retail_amit_vaswani,user_updates,2024-05-15T17:00:19.295Z,amit.vaswani@databricks.com,amit.vaswani@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/a273ddbf-9b45-4bed-9cbb-f386d88b5aa6,null,null,2024-05-15T17:00:19.295Z,2024-05-15T18:00:44Z,0.006897926330566406,null,null,null,,,"[{""size"": 0.006897926330566406, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,marzi_pdf_poc,pdf_extract_tbl_pgwise_2,2024-05-13T14:10:19.876Z,marzi.rasooli@databricks.com,marzi.rasooli@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/56320285-c66c-4516-ad0e-d424e3c6574f,null,null,2024-05-13T14:10:19.876Z,null,0.0571441650390625,null,null,null,,,"[{""size"": 0.0571441650390625, ""date"": ""2024-06-03""}]"
2024-06-03T00:00:00Z,dbdemos,rag_chatbot_sal_deloera_newco,pdf_raw,2024-05-15T18:12:15.981Z,sal.deloera@databricks.com,sal.deloera@databricks.com,MANAGED,DELTA,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/9bbc1269-dad4-46e3-9303-9fe86ec8461d,null,null,2024-05-15T18:12:15.981Z,null,10.449095726013184,null,null,null,,,"[{""size"": 10.449095726

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
spark.sql(f"DROP TABLE {temporary_table}")

DataFrame[]